In [ ]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path  # 新增：用于处理目录路径
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# -------------------------- 预计算并存储关键结果 --------------------------
def precompute_and_save(data_path, save_dir):
    """
    预计算关键数据并存储：
    1. 股票基础数据（含指标）：stock_data_with_indicators.csv（改为CSV格式，避免Parquet依赖问题）
    2. 每日符合条件股票：daily_eligible_stocks.pkl
    3. 有效股票列表：valid_stocks.pkl
    4. 全市场交易日：market_days.pkl
    """
    # 关键修复：确保存储目录存在，不存在则创建
    save_dir = Path(save_dir)
    save_dir.mkdir(exist_ok=True, parents=True)  # 自动创建目录（包括父目录）
    
    # 1. 读取原始数据
    if data_path.endswith('.parquet'):
        df = pd.read_parquet(data_path, engine="pyarrow")
    else:
        df = pd.read_csv(data_path)
    
    # 2. 基础处理（同原逻辑）
    needed_cols = ["date", "stock_code", "open", "close", "volume", "pre_close",
                   "high_limit", "low_limit", "paused"]
    missing_cols = [col for col in needed_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"数据缺少必要字段：{missing_cols}")
    
    df["date"] = pd.to_datetime(df["date"])
    df = df[(df["date"] >= "2023-01-01") & (df["date"] <= "2025-10-27")]
    df["stock_code"] = df["stock_code"].astype(str).str.strip()
    
    # 3. 精简股票（按日均成交量前500只筛选）
    stock_daily_volume = df.groupby("stock_code")["volume"].mean()
    valid_stocks = stock_daily_volume.nlargest(5000).index.tolist()
    df = df[df["stock_code"].isin(valid_stocks)]
    
    # 4. 数据清洗+指标计算
    df = df[
        (df["volume"] > 100) &
        (df["open"] > 0.01) &
        (df["close"] > 0.01)
    ].reset_index(drop=True)
    
    df["paused"] = df["paused"].fillna(1.0).astype("float32")
    df["price_change"] = (df["close"] - df["pre_close"]) / df["pre_close"] * 100
    df["is_limit_up"] = (df["close"] >= df["high_limit"] * 0.99) & (df["price_change"] >= 9.0)
    df["vol_3d_avg"] = df.groupby("stock_code", observed=True)["volume"].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True)
    df["volume_ratio"] = df["volume"] / df["vol_3d_avg"].clip(lower=1)
    
    # 5. 预计算每日符合条件股票
    daily_eligible = {}
    market_days = sorted(df["date"].unique())
    for date in tqdm(market_days, desc="预计算每日符合条件股票"):
        date_df = df[df["date"] == date]
        if len(date_df) < 2:
            daily_eligible[date] = []
            continue
        eligible = date_df[
            (date_df["volume_ratio"] > 0.8) &
            ((date_df["is_limit_up"]) | (date_df["price_change"] > 1))
        ].sort_values("price_change", ascending=False)["stock_code"].tolist()
        daily_eligible[date] = eligible[:4]
    
    # 6. 存储结果（改用CSV格式存储股票数据，避免Parquet可能的依赖问题）
    # - 股票数据（含指标）：CSV格式（兼容性更好）
    df.to_csv(f"{save_dir}/stock_data_with_indicators.csv", index=False, encoding="utf-8-sig")
    # - 字典/列表：pickle格式
    with open(f"{save_dir}/daily_eligible_stocks.pkl", "wb") as f:
        pickle.dump(daily_eligible, f)
    with open(f"{save_dir}/valid_stocks.pkl", "wb") as f:
        pickle.dump(valid_stocks, f)
    with open(f"{save_dir}/market_days.pkl", "wb") as f:
        pickle.dump(market_days, f)
    
    print(f"预计算完成！文件已保存至：{save_dir}")
    print(f" - 有效股票数：{len(valid_stocks)}")
    print(f" - 全市场交易日数：{len(market_days)}")

# -------------------------- 运行预计算 --------------------------
if __name__ == "__main__":
    # 配置路径（请根据实际情况修改）
    RAW_DATA_PATH = "D:\\workspace\\xiaoyao\\data\\widetable.parquet"  # 原始数据路径
    SAVE_DIR = ".\\precomputed_data"  # 存储目录（相对路径，会自动创建）
    
    # 执行预计算
    precompute_and_save(RAW_DATA_PATH, SAVE_DIR)

预计算每日符合条件股票: 100%|██████████| 680/680 [00:02<00:00, 228.99it/s]


预计算完成！文件已保存至：precomputed_data
 - 有效股票数：500
 - 全市场交易日数：680
